# CLUSTERING OF DISTRICTS IN GUJARAT FOR REAL ESTATE INVESTMENTS

# Assignment by: Shivam Awasthi

In this Assignment, I will try to cluster the districts of Gujarat to various clusters based on their urban development and Population so that ABC group(who is a Real Estate Giant will have the sufficient insights to plan strategic investments in Gujarat)

Installing and importing the required libraries

In [1]:
!pip install geopy
from geopy.geocoders import Nominatim
import json # library to handle JSON files

#!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
#from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans
#!conda install -c conda-forge folium=0.5.0 --yes
!pip install folium
#!pip install -c conda-forge folium=0.5.0--yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library
from sklearn import preprocessing
print('Libraries imported.')

     |████████████████████████████████| 102kB 6.8MB/s ta 0:00:011
Libraries imported.


# Webscraping the required districts data from wikipedia 

In [2]:
import pandas as pd
url = 'https://en.wikipedia.org/wiki/List_of_districts_of_Gujarat'
dfs = pd.read_html(url)

print(len(dfs))
dfs[1]

5


,No.,District,DistrictHeadquarters,Population2001 Census[5],Population2011 Census[5],Area (km²),Density (per km²)2011,Year ofFormation,Taluka/Tehsil,TotalTalukas
0,1,Ahmedabad,Ahmedabad,5673090,7045313,7170,983,1960,City East City WestBavlaDaskroiDetroj-RampuraD...,10
1,2,Amreli,Amreli,1393880,1513614,6760,224,1960,AmreliBabraBagasaraDhariJafrabadKhambhaKunkava...,11
2,3,Anand,Anand,1856712,2090276,4690,446,1997,AnandAnklavBorsadKhambhatPetladSojitraTarapurU...,8
3,4,Aravalli,Modasa,908797,1039918,3217,323,2013,BayadBhilodaDhansuraMalpurMeghrajModasa,6
4,5,Banaskantha,Palanpur,2502843,3116045,12703,245,1960,AmirgadhBhabharDantaDantiwadaDeesaDeodarDhaner...,14
5,6,Bharuch,Bharuch,1370104,1550822,6524,238,1960,BharuchAmodAnkleshwarHansotJambusarJhagadiaNet...,9
6,7,Bhavnagar,Bhavnagar,2065492,2393272,8334,287,1960,BhavnagarGariadharGhoghaJesarMahuvaPalitanaSih...,10
7,8,Botad,Botad,547567,656005,2564,256,2013,BotadBarwalaGadhadaRanpur,4
8,9,Chhota Udaipur,Chhota Udaipur,909799,1071831,3237,331,2013,Chhota UdepurBodeliJetpur paviKavantNasvadiSan...,6
9,10,Dahod,Dahod,1635374,2126558,3643,583,1997,DahodDevgadh bariaDhanpurFatepuraGarbadaLimkhe...,9


Cleaning the table

In [3]:
df=dfs[1]
df.drop(df.columns[[2,5,7,8,9]], axis = 1, inplace = True)
df.drop([33], inplace = True )
df.set_index('No.')
df.columns = ['No.', 'District', 'Population2001', 'Population2011', 
                'Density2011'] 
df.head()

,No.,District,Population2001,Population2011,Density2011
0,1,Ahmedabad,5673090,7045313,983
1,2,Amreli,1393880,1513614,224
2,3,Anand,1856712,2090276,446
3,4,Aravalli,908797,1039918,323
4,5,Banaskantha,2502843,3116045,245


Adding a new column for Population growth percentage

In [4]:
df['PopulationGrowth'] = " "
df[['Population2001', 'Population2011','Density2011']] = df[['Population2001', 'Population2011','Density2011']].apply(pd.to_numeric) 
for i in range(len(df)):
    df['PopulationGrowth']=100*(df['Population2011']-df['Population2001'])/df['Population2001']
df.head()


,No.,District,Population2001,Population2011,Density2011,PopulationGrowth
0,1,Ahmedabad,5673090,7045313,983,24.188282
1,2,Amreli,1393880,1513614,224,8.589979
2,3,Anand,1856712,2090276,446,12.579442
3,4,Aravalli,908797,1039918,323,14.427975
4,5,Banaskantha,2502843,3116045,245,24.500218


# Extracting the Location data using Goecoder

In [6]:
df['Latitude']=None
df['Longitude']=None
df['District&state'] = df['District'].astype(str) + ', Gujarat'
df.head()

,No.,District,Population2001,Population2011,Density2011,PopulationGrowth,Latitude,Longitude,District&state
0,1,Ahmedabad,5673090,7045313,983,24.188282,None,None,"Ahmedabad, Gujarat"
1,2,Amreli,1393880,1513614,224,8.589979,None,None,"Amreli, Gujarat"
2,3,Anand,1856712,2090276,446,12.579442,None,None,"Anand, Gujarat"
3,4,Aravalli,908797,1039918,323,14.427975,None,None,"Aravalli, Gujarat"
4,5,Banaskantha,2502843,3116045,245,24.500218,None,None,"Banaskantha, Gujarat"


In [7]:
geolocator = Nominatim(user_agent="foursquare_agent")
for i in range(len(df)):
    location = geolocator.geocode(df['District&state'][i])
    df['Latitude'][i]=location.latitude if geolocator.geocode(df['District&state'][i]) != None else 'NF'
    df['Longitude'][i]=location.longitude if geolocator.geocode(df['District&state'][i]) != None else 'NF'
df

/opt/conda/envs/Python36/lib/python3.6/site-packages/ipykernel/__main__.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/opt/conda/envs/Python36/lib/python3.6/site-packages/ipykernel/__main__.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


,No.,District,Population2001,Population2011,Density2011,PopulationGrowth,Latitude,Longitude,District&state
0,1,Ahmedabad,5673090,7045313,983,24.188282,23.0216,72.5797,"Ahmedabad, Gujarat"
1,2,Amreli,1393880,1513614,224,8.589979,20.8667,70.75,"Amreli, Gujarat"
2,3,Anand,1856712,2090276,446,12.579442,22.5585,72.9626,"Anand, Gujarat"
3,4,Aravalli,908797,1039918,323,14.427975,23.4835,73.3988,"Aravalli, Gujarat"
4,5,Banaskantha,2502843,3116045,245,24.500218,24.1721,72.4311,"Banaskantha, Gujarat"
5,6,Bharuch,1370104,1550822,238,13.190094,21.75,73,"Bharuch, Gujarat"
6,7,Bhavnagar,2065492,2393272,287,15.869343,21.7719,72.1416,"Bhavnagar, Gujarat"
7,8,Botad,547567,656005,256,19.803604,22.1686,71.6685,"Botad, Gujarat"
8,9,Chhota Udaipur,909799,1071831,331,17.809648,22.3049,74.0126,"Chhota Udaipur, Gujarat"
9,10,Dahod,1635374,2126558,583,30.034964,22.8359,74.2557,"Dahod, Gujarat"


In [8]:
df['Latitude'][11] = 22.12   #Manually adding data for Devbhoomi Dwarka
df['Longitude'][11] = 69.39

/opt/conda/envs/Python36/lib/python3.6/site-packages/ipykernel/__main__.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':
/opt/conda/envs/Python36/lib/python3.6/site-packages/ipykernel/__main__.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  from ipykernel import kernelapp as app


This is the final data required for Analysis

In [9]:
df.drop(df.columns[[8]], axis = 1, inplace = True)
df1=df.copy()
df.head()

,No.,District,Population2001,Population2011,Density2011,PopulationGrowth,Latitude,Longitude
0,1,Ahmedabad,5673090,7045313,983,24.188282,23.0216,72.5797
1,2,Amreli,1393880,1513614,224,8.589979,20.8667,70.75
2,3,Anand,1856712,2090276,446,12.579442,22.5585,72.9626
3,4,Aravalli,908797,1039918,323,14.427975,23.4835,73.3988
4,5,Banaskantha,2502843,3116045,245,24.500218,24.1721,72.4311


# Using Foursquare Data To explore the Venues in these Districts

In [10]:
address = 'Gujarat, India'
loc = geolocator.geocode(address)
latitude = loc.latitude
longitude = loc.longitude
print(latitude, longitude)

22.3850008 71.7452633


In [11]:
CLIENT_ID = '5GKE0APQRCVPYSSI4B5JW0KFJFLOMSX4WWV0ZXULZQDD4MBK' # my Foursquare ID
CLIENT_SECRET = 'HL3O0HK0J1ZUCISELTMKEA1BKMVXCQGBN1THMIIANURPHZ44' # my Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('My credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

My credentails:
CLIENT_ID: 5GKE0APQRCVPYSSI4B5JW0KFJFLOMSX4WWV0ZXULZQDD4MBK
CLIENT_SECRET:HL3O0HK0J1ZUCISELTMKEA1BKMVXCQGBN1THMIIANURPHZ44


In [12]:
LIMIT = 100

In [13]:
def getNearbyVenues(names, latitudes, longitudes, radius=5000):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])
        nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['District', 
                  'District Latitude', 
                  'District Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [14]:
Gujarat_venues = getNearbyVenues(names=df['District'],
                                   latitudes=df['Latitude'],
                                   longitudes=df['Longitude'])

Ahmedabad
Amreli
Anand
Aravalli
Banaskantha
Bharuch
Bhavnagar
Botad
Chhota Udaipur
Dahod
Dang
Devbhoomi Dwarka
Gandhinagar
Gir Somnath
Jamnagar
Junagadh
Kutch
Kheda
Mahisagar
Mehsana
Morbi
Narmada
Navsari
Panchmahal
Patan
Porbandar
Rajkot
Sabarkantha
Surat
Surendranagar
Tapi
Vadodara
Valsad


In [15]:
print(Gujarat_venues.shape)
Gujarat_venues.head()

(446, 7)


,District,District Latitude,District Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Ahmedabad,23.021624,72.579707,Manek Chowk Khau Gali,23.023505,72.588539,Snack Place
1,Ahmedabad,23.021624,72.579707,Manek Chowk,23.023626,72.588553,Fast Food Restaurant
2,Ahmedabad,23.021624,72.579707,Lucky Tea,23.027829,72.581394,Tea Room
3,Ahmedabad,23.021624,72.579707,Agashiye,23.027104,72.581614,Indian Restaurant
4,Ahmedabad,23.021624,72.579707,Crossword,23.032656,72.565404,Bookstore


Cleaning the Venue data for Analysis

In [16]:
Gujarat_venues.groupby('District').count()
# one hot encoding
Gujarat_onehot = pd.get_dummies(Gujarat_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
Gujarat_onehot['District'] = Gujarat_venues['District'] 

# move neighborhood column to the first column
fixed_columns = [Gujarat_onehot.columns[-1]] + list(Gujarat_onehot.columns[:-1])
Gujarat_onehot = Gujarat_onehot[fixed_columns]

Gujarat_onehot.head()

,District,ATM,Airport,Airport Terminal,American Restaurant,Arcade,Art Gallery,Arts & Crafts Store,Arts & Entertainment,Asian Restaurant,...,Snack Place,Stadium,Supermarket,Tea Room,Theater,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Store,Zoo
0,Ahmedabad,0,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
1,Ahmedabad,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,Ahmedabad,0,0,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0
3,Ahmedabad,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,Ahmedabad,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [17]:
Gujarat_grouped = Gujarat_onehot.groupby('District').mean().reset_index()
Gujarat_grouped

,District,ATM,Airport,Airport Terminal,American Restaurant,Arcade,Art Gallery,Arts & Crafts Store,Arts & Entertainment,Asian Restaurant,...,Snack Place,Stadium,Supermarket,Tea Room,Theater,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Store,Zoo
0,Ahmedabad,0.0,0.000000,0.000000,0.00,0.010526,0.010526,0.010526,0.00,0.010526,...,0.031579,0.000000,0.000000,0.052632,0.010526,0.00,0.010526,0.031579,0.010526,0.010526
1,Amreli,0.0,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.00,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000
2,Anand,0.0,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.00,0.050000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000
3,Banaskantha,0.0,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.00,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.166667,0.000000,0.000000,0.000000
4,Bharuch,0.0,0.000000,0.000000,0.25,0.000000,0.000000,0.000000,0.00,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000
5,Bhavnagar,0.0,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.00,0.100000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000
6,Botad,0.0,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.00,0.111111,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.111111,0.000000,0.000000,0.000000
7,Chhota Udaipur,0.5,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.00,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000
8,Dahod,0.0,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.00,0.250000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.250000,0.000000,0.000000,0.000000
9,Gandhinagar,0.0,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.04,0.080000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.04,0.000000,0.000000,0.000000,0.000000


In [18]:
Gujarat_grouped1=Gujarat_grouped.copy()

Finding the top 10 venues for exploring the district

In [19]:
num_top_venues = 10

for dist in Gujarat_grouped['District']:
    print("----"+dist+"----")
    temp = Gujarat_grouped[Gujarat_grouped['District'] == dist].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Ahmedabad----
                           venue  freq
0              Indian Restaurant  0.15
1                           Café  0.12
2                          Hotel  0.08
3           Fast Food Restaurant  0.07
4                       Tea Room  0.05
5                   Dessert Shop  0.03
6  Vegetarian / Vegan Restaurant  0.03
7                    Snack Place  0.03
8                 Sandwich Place  0.03
9                          Diner  0.02


----Amreli----
                 venue  freq
0                Beach   1.0
1                  ATM   0.0
2                 Park   0.0
3            Multiplex   0.0
4        Movie Theater   0.0
5             Mountain   0.0
6                Motel   0.0
7  Moroccan Restaurant   0.0
8    Mobile Phone Shop   0.0
9   Miscellaneous Shop   0.0


----Anand----
                  venue  freq
0  Fast Food Restaurant  0.15
1     Indian Restaurant  0.10
2                  Café  0.10
3          Dessert Shop  0.10
4        Sandwich Place  0.05
5            Restaura

In [20]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [21]:
import numpy as np
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['District']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
District_venues_sorted = pd.DataFrame(columns=columns)
District_venues_sorted['District'] = Gujarat_grouped['District']

for ind in np.arange(Gujarat_grouped.shape[0]):
    District_venues_sorted.iloc[ind, 1:] = return_most_common_venues(Gujarat_grouped.iloc[ind, :], num_top_venues)

District_venues_sorted.head()

,District,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Ahmedabad,Indian Restaurant,Café,Hotel,Fast Food Restaurant,Tea Room,Sandwich Place,Vegetarian / Vegan Restaurant,Dessert Shop,Snack Place,History Museum
1,Amreli,Beach,Zoo,Clothing Store,Construction & Landscaping,Cricket Ground,Dairy Store,Department Store,Dessert Shop,Diner,Electronics Store
2,Anand,Fast Food Restaurant,Dessert Shop,Indian Restaurant,Café,Resort,Restaurant,Pizza Place,Sandwich Place,Factory,BBQ Joint
3,Banaskantha,Indian Restaurant,Train Station,Mobile Phone Shop,Ice Cream Shop,Multiplex,Zoo,Electronics Store,Coffee Shop,Construction & Landscaping,Cricket Ground
4,Bharuch,Hotel,Multiplex,American Restaurant,Factory,Coffee Shop,Construction & Landscaping,Cricket Ground,Dairy Store,Department Store,Dessert Shop


In [22]:
District_venues_sorted1=District_venues_sorted.copy()

# Clustering with respect to Urban Development(No. of Clusters=4)

In [23]:
# set number of clusters
kclusters = 4

Gujarat_grouped_clustering = Gujarat_grouped.drop('District', 1)

kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(Gujarat_grouped_clustering)

kmeans.labels_[0:10]

array([0, 3, 0, 0, 2, 0, 0, 1, 0, 0], dtype=int32)

In [24]:
 #add clustering labels
District_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

Gujarat_merged = df

Gujarat_merged = Gujarat_merged.join(District_venues_sorted.set_index('District'), on='District')

Gujarat_merged 

,No.,District,Population2001,Population2011,Density2011,PopulationGrowth,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,1,Ahmedabad,5673090,7045313,983,24.188282,23.0216,72.5797,0.0,Indian Restaurant,Café,Hotel,Fast Food Restaurant,Tea Room,Sandwich Place,Vegetarian / Vegan Restaurant,Dessert Shop,Snack Place,History Museum
1,2,Amreli,1393880,1513614,224,8.589979,20.8667,70.75,3.0,Beach,Zoo,Clothing Store,Construction & Landscaping,Cricket Ground,Dairy Store,Department Store,Dessert Shop,Diner,Electronics Store
2,3,Anand,1856712,2090276,446,12.579442,22.5585,72.9626,0.0,Fast Food Restaurant,Dessert Shop,Indian Restaurant,Café,Resort,Restaurant,Pizza Place,Sandwich Place,Factory,BBQ Joint
3,4,Aravalli,908797,1039918,323,14.427975,23.4835,73.3988,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,5,Banaskantha,2502843,3116045,245,24.500218,24.1721,72.4311,0.0,Indian Restaurant,Train Station,Mobile Phone Shop,Ice Cream Shop,Multiplex,Zoo,Electronics Store,Coffee Shop,Construction & Landscaping,Cricket Ground
5,6,Bharuch,1370104,1550822,238,13.190094,21.75,73,2.0,Hotel,Multiplex,American Restaurant,Factory,Coffee Shop,Construction & Landscaping,Cricket Ground,Dairy Store,Department Store,Dessert Shop
6,7,Bhavnagar,2065492,2393272,287,15.869343,21.7719,72.1416,0.0,Café,Fast Food Restaurant,Multiplex,Garden,Hotel,Pizza Place,Asian Restaurant,Shopping Mall,Diner,Coffee Shop
7,8,Botad,547567,656005,256,19.803604,22.1686,71.6685,0.0,Restaurant,Bakery,Chinese Restaurant,Train Station,Bus Station,Pub,Asian Restaurant,Jewelry Store,Cricket Ground,Dairy Store
8,9,Chhota Udaipur,909799,1071831,331,17.809648,22.3049,74.0126,1.0,ATM,Ice Cream Shop,Farmers Market,Construction & Landscaping,Cricket Ground,Dairy Store,Department Store,Dessert Shop,Diner,Electronics Store
9,10,Dahod,1635374,2126558,583,30.034964,22.8359,74.2557,0.0,Train Station,Motel,Indian Restaurant,Asian Restaurant,Zoo,Factory,Construction & Landscaping,Cricket Ground,Dairy Store,Department Store


In [25]:
for i in range(len(Gujarat_merged)):
    if (np.isnan(Gujarat_merged['Cluster Labels'][i])):
        print(Gujarat_merged['District'][i]) 

Aravalli
Dang
Devbhoomi Dwarka
Gir Somnath
Kutch
Mahisagar
Narmada
Patan


These above Districts are least in Urban Development as they did not have enough centers, Hence ,these districts(Labelling them as Cluster -1) are ignored from the further analysis

In [26]:
Gujarat_merged = Gujarat_merged[Gujarat_merged['Cluster Labels'].notna()]
Gujarat_merged['Cluster Labels']=Gujarat_merged['Cluster Labels'].astype(int)
Gujarat_merged.head()

/opt/conda/envs/Python36/lib/python3.6/site-packages/ipykernel/__main__.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  from ipykernel import kernelapp as app


,No.,District,Population2001,Population2011,Density2011,PopulationGrowth,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,1,Ahmedabad,5673090,7045313,983,24.188282,23.0216,72.5797,0,Indian Restaurant,Café,Hotel,Fast Food Restaurant,Tea Room,Sandwich Place,Vegetarian / Vegan Restaurant,Dessert Shop,Snack Place,History Museum
1,2,Amreli,1393880,1513614,224,8.589979,20.8667,70.75,3,Beach,Zoo,Clothing Store,Construction & Landscaping,Cricket Ground,Dairy Store,Department Store,Dessert Shop,Diner,Electronics Store
2,3,Anand,1856712,2090276,446,12.579442,22.5585,72.9626,0,Fast Food Restaurant,Dessert Shop,Indian Restaurant,Café,Resort,Restaurant,Pizza Place,Sandwich Place,Factory,BBQ Joint
4,5,Banaskantha,2502843,3116045,245,24.500218,24.1721,72.4311,0,Indian Restaurant,Train Station,Mobile Phone Shop,Ice Cream Shop,Multiplex,Zoo,Electronics Store,Coffee Shop,Construction & Landscaping,Cricket Ground
5,6,Bharuch,1370104,1550822,238,13.190094,21.75,73,2,Hotel,Multiplex,American Restaurant,Factory,Coffee Shop,Construction & Landscaping,Cricket Ground,Dairy Store,Department Store,Dessert Shop


Viewing the Clusters on the map

In [35]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=7)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(Gujarat_merged['Latitude'], Gujarat_merged['Longitude'], Gujarat_merged['District'], Gujarat_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-3],
        fill=True,
        fill_color=rainbow[cluster-2],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

# Clustering with Respect to Urban Development(No. of clusters=3)

In [101]:
# set number of clusters
kclusters = 3
Gujarat_grouped2=Gujarat_grouped1.copy()
Gujarat_grouped_clustering2 = Gujarat_grouped2.drop('District', 1)

kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(Gujarat_grouped_clustering2)

kmeans.labels_[0:10]

array([0, 2, 0, 0, 0, 0, 0, 1, 0, 0], dtype=int32)

In [102]:
#add clustering labels
District_venues_sorted2=District_venues_sorted1.copy()
District_venues_sorted2.insert(0, 'Cluster Labels', kmeans.labels_)
df2=df1.copy()
Gujarat_merged2 = df2

Gujarat_merged2 = Gujarat_merged2.join(District_venues_sorted2.set_index('District'), on='District')

Gujarat_merged2

,No.,District,Population2001,Population2011,Density2011,PopulationGrowth,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,1,Ahmedabad,5673090,7045313,983,24.188282,23.0216,72.5797,0.0,Indian Restaurant,Café,Hotel,Fast Food Restaurant,Tea Room,Sandwich Place,Vegetarian / Vegan Restaurant,Dessert Shop,Snack Place,History Museum
1,2,Amreli,1393880,1513614,224,8.589979,20.8667,70.75,2.0,Beach,Zoo,Clothing Store,Construction & Landscaping,Cricket Ground,Dairy Store,Department Store,Dessert Shop,Diner,Electronics Store
2,3,Anand,1856712,2090276,446,12.579442,22.5585,72.9626,0.0,Fast Food Restaurant,Dessert Shop,Indian Restaurant,Café,Resort,Restaurant,Pizza Place,Sandwich Place,Factory,BBQ Joint
3,4,Aravalli,908797,1039918,323,14.427975,23.4835,73.3988,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,5,Banaskantha,2502843,3116045,245,24.500218,24.1721,72.4311,0.0,Indian Restaurant,Train Station,Mobile Phone Shop,Ice Cream Shop,Multiplex,Zoo,Electronics Store,Coffee Shop,Construction & Landscaping,Cricket Ground
5,6,Bharuch,1370104,1550822,238,13.190094,21.75,73,0.0,Hotel,Multiplex,American Restaurant,Factory,Coffee Shop,Construction & Landscaping,Cricket Ground,Dairy Store,Department Store,Dessert Shop
6,7,Bhavnagar,2065492,2393272,287,15.869343,21.7719,72.1416,0.0,Café,Fast Food Restaurant,Multiplex,Garden,Hotel,Pizza Place,Asian Restaurant,Shopping Mall,Diner,Coffee Shop
7,8,Botad,547567,656005,256,19.803604,22.1686,71.6685,0.0,Restaurant,Bakery,Chinese Restaurant,Train Station,Bus Station,Pub,Asian Restaurant,Jewelry Store,Cricket Ground,Dairy Store
8,9,Chhota Udaipur,909799,1071831,331,17.809648,22.3049,74.0126,1.0,ATM,Ice Cream Shop,Farmers Market,Construction & Landscaping,Cricket Ground,Dairy Store,Department Store,Dessert Shop,Diner,Electronics Store
9,10,Dahod,1635374,2126558,583,30.034964,22.8359,74.2557,0.0,Train Station,Motel,Indian Restaurant,Asian Restaurant,Zoo,Factory,Construction & Landscaping,Cricket Ground,Dairy Store,Department Store


In [103]:
Gujarat_merged2 = Gujarat_merged2[Gujarat_merged2['Cluster Labels'].notna()]
Gujarat_merged2['Cluster Labels']=Gujarat_merged2['Cluster Labels'].astype(int)
Gujarat_merged2.head()

/opt/conda/envs/Python36/lib/python3.6/site-packages/ipykernel/__main__.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  from ipykernel import kernelapp as app


,No.,District,Population2001,Population2011,Density2011,PopulationGrowth,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,1,Ahmedabad,5673090,7045313,983,24.188282,23.0216,72.5797,0,Indian Restaurant,Café,Hotel,Fast Food Restaurant,Tea Room,Sandwich Place,Vegetarian / Vegan Restaurant,Dessert Shop,Snack Place,History Museum
1,2,Amreli,1393880,1513614,224,8.589979,20.8667,70.75,2,Beach,Zoo,Clothing Store,Construction & Landscaping,Cricket Ground,Dairy Store,Department Store,Dessert Shop,Diner,Electronics Store
2,3,Anand,1856712,2090276,446,12.579442,22.5585,72.9626,0,Fast Food Restaurant,Dessert Shop,Indian Restaurant,Café,Resort,Restaurant,Pizza Place,Sandwich Place,Factory,BBQ Joint
4,5,Banaskantha,2502843,3116045,245,24.500218,24.1721,72.4311,0,Indian Restaurant,Train Station,Mobile Phone Shop,Ice Cream Shop,Multiplex,Zoo,Electronics Store,Coffee Shop,Construction & Landscaping,Cricket Ground
5,6,Bharuch,1370104,1550822,238,13.190094,21.75,73,0,Hotel,Multiplex,American Restaurant,Factory,Coffee Shop,Construction & Landscaping,Cricket Ground,Dairy Store,Department Store,Dessert Shop


In [104]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=7)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(Gujarat_merged2['Latitude'], Gujarat_merged2['Longitude'], Gujarat_merged2['District'], Gujarat_merged2['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-3],
        fill=True,
        fill_color=rainbow[cluster-2],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

# Clustering with Respect to Urban Development and Population Growth(No. of Clusters = 3)

In [139]:
Gujarat_grouped4=Gujarat_grouped1.copy()
District_venues_sorted4=District_venues_sorted1.copy()
df4=df1.copy()

In [140]:
Gujarat_grouped4['Population2011']=None
Gujarat_grouped4['Density2011']=None
Gujarat_grouped4['PopulationGrowth']=None

for i in range(len(Gujarat_grouped4)):
    for j in range(len(df4)):
        if (Gujarat_grouped4['District'][i] == df4['District'][j]):
            Gujarat_grouped4['Population2011'][i]=df4['Population2011'][j]
            Gujarat_grouped4['Density2011'][i]=df4['Density2011'][j]
            Gujarat_grouped4['PopulationGrowth'][i]=df4['PopulationGrowth'][j]

/opt/conda/envs/Python36/lib/python3.6/site-packages/ipykernel/__main__.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/opt/conda/envs/Python36/lib/python3.6/site-packages/ipykernel/__main__.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/opt/conda/envs/Python36/lib/python3.6/site-packages/ipykernel/__main__.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [141]:
Gujarat_grouped4.head()

,District,ATM,Airport,Airport Terminal,American Restaurant,Arcade,Art Gallery,Arts & Crafts Store,Arts & Entertainment,Asian Restaurant,...,Tea Room,Theater,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Store,Zoo,Population2011,Density2011,PopulationGrowth
0,Ahmedabad,0.0,0.0,0.0,0.00,0.010526,0.010526,0.010526,0.0,0.010526,...,0.052632,0.010526,0.0,0.010526,0.031579,0.010526,0.010526,7045313,983,24.1883
1,Amreli,0.0,0.0,0.0,0.00,0.000000,0.000000,0.000000,0.0,0.000000,...,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,1513614,224,8.58998
2,Anand,0.0,0.0,0.0,0.00,0.000000,0.000000,0.000000,0.0,0.050000,...,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,2090276,446,12.5794
3,Banaskantha,0.0,0.0,0.0,0.00,0.000000,0.000000,0.000000,0.0,0.000000,...,0.000000,0.000000,0.0,0.166667,0.000000,0.000000,0.000000,3116045,245,24.5002
4,Bharuch,0.0,0.0,0.0,0.25,0.000000,0.000000,0.000000,0.0,0.000000,...,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,1550822,238,13.1901


In [142]:
Gujarat_grouped_clustering1 = Gujarat_grouped4.drop('District', 1)
Gujarat_grouped_clustering1= preprocessing.StandardScaler().fit(Gujarat_grouped_clustering1).transform(Gujarat_grouped_clustering1)
Gujarat_grouped_clustering1[0:5]

/opt/conda/envs/Python36/lib/python3.6/site-packages/sklearn/preprocessing/data.py:645: DataConversionWarning: Data with input dtype float64, object were all converted to float64 by StandardScaler.
  return self.partial_fit(X, y)
/opt/conda/envs/Python36/lib/python3.6/site-packages/ipykernel/__main__.py:2: DataConversionWarning: Data with input dtype float64, object were all converted to float64 by StandardScaler.
  from ipykernel import kernelapp as app


array([[-3.53553391e-01, -2.64044451e-01, -2.04124145e-01,
        -2.04124145e-01,  4.89897949e+00,  4.89897949e+00,
         3.41041287e+00, -2.04124145e-01, -4.31494451e-01,
        -2.04124145e-01, -2.86702709e-01, -1.12132989e-01,
        -2.42140294e-01,  3.41041287e+00, -2.04124145e-01,
        -2.04124145e-01, -2.94330928e-01, -2.04124145e-01,
        -2.04124145e-01,  1.53072316e+00, -3.10886849e-01,
        -2.04124145e-01,  1.47826620e+00,  9.41582893e-01,
        -2.04124145e-01,  4.89897949e+00, -2.04124145e-01,
         6.66854636e-01,  1.16424019e+00,  4.09069174e+00,
        -2.75088043e-01, -2.04124145e-01,  4.89897949e+00,
         4.44303831e-01, -2.04124145e-01, -2.87863977e-01,
        -2.04124145e-01, -2.04124145e-01, -2.04124145e-01,
        -2.04124145e-01, -2.04124145e-01,  3.41041287e+00,
        -2.04124145e-01,  4.37340697e+00,  4.37340697e+00,
        -2.04124145e-01, -8.73878827e-02,  3.41041287e+00,
        -2.14222606e-01,  2.99502994e-01, -2.04124145e-0

In [143]:
kclusters=3
# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(Gujarat_grouped_clustering1)

# check cluster labels generated for each row in the dataframe
kmeans.labels_

array([0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 1,
       1, 1, 1], dtype=int32)

In [144]:
#add clustering labels
District_venues_sorted4=District_venues_sorted1.copy()
District_venues_sorted4.insert(0, 'Cluster Labels', kmeans.labels_)
Gujarat_merged4 = df4

Gujarat_merged4 = Gujarat_merged4.join(District_venues_sorted4.set_index('District'), on='District')

Gujarat_merged4.head()

,No.,District,Population2001,Population2011,Density2011,PopulationGrowth,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,1,Ahmedabad,5673090,7045313,983,24.188282,23.0216,72.5797,0.0,Indian Restaurant,Café,Hotel,Fast Food Restaurant,Tea Room,Sandwich Place,Vegetarian / Vegan Restaurant,Dessert Shop,Snack Place,History Museum
1,2,Amreli,1393880,1513614,224,8.589979,20.8667,70.75,1.0,Beach,Zoo,Clothing Store,Construction & Landscaping,Cricket Ground,Dairy Store,Department Store,Dessert Shop,Diner,Electronics Store
2,3,Anand,1856712,2090276,446,12.579442,22.5585,72.9626,1.0,Fast Food Restaurant,Dessert Shop,Indian Restaurant,Café,Resort,Restaurant,Pizza Place,Sandwich Place,Factory,BBQ Joint
3,4,Aravalli,908797,1039918,323,14.427975,23.4835,73.3988,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,5,Banaskantha,2502843,3116045,245,24.500218,24.1721,72.4311,1.0,Indian Restaurant,Train Station,Mobile Phone Shop,Ice Cream Shop,Multiplex,Zoo,Electronics Store,Coffee Shop,Construction & Landscaping,Cricket Ground


In [145]:
Gujarat_merged4 = Gujarat_merged4[Gujarat_merged4['Cluster Labels'].notna()]
Gujarat_merged4['Cluster Labels']=Gujarat_merged4['Cluster Labels'].astype(int)
Gujarat_merged4.head()

,No.,District,Population2001,Population2011,Density2011,PopulationGrowth,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,1,Ahmedabad,5673090,7045313,983,24.188282,23.0216,72.5797,0,Indian Restaurant,Café,Hotel,Fast Food Restaurant,Tea Room,Sandwich Place,Vegetarian / Vegan Restaurant,Dessert Shop,Snack Place,History Museum
1,2,Amreli,1393880,1513614,224,8.589979,20.8667,70.75,1,Beach,Zoo,Clothing Store,Construction & Landscaping,Cricket Ground,Dairy Store,Department Store,Dessert Shop,Diner,Electronics Store
2,3,Anand,1856712,2090276,446,12.579442,22.5585,72.9626,1,Fast Food Restaurant,Dessert Shop,Indian Restaurant,Café,Resort,Restaurant,Pizza Place,Sandwich Place,Factory,BBQ Joint
4,5,Banaskantha,2502843,3116045,245,24.500218,24.1721,72.4311,1,Indian Restaurant,Train Station,Mobile Phone Shop,Ice Cream Shop,Multiplex,Zoo,Electronics Store,Coffee Shop,Construction & Landscaping,Cricket Ground
5,6,Bharuch,1370104,1550822,238,13.190094,21.75,73,1,Hotel,Multiplex,American Restaurant,Factory,Coffee Shop,Construction & Landscaping,Cricket Ground,Dairy Store,Department Store,Dessert Shop


In [146]:
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=7)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(Gujarat_merged4['Latitude'], Gujarat_merged4['Longitude'], Gujarat_merged4['District'], Gujarat_merged4['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-3],
        fill=True,
        fill_color=rainbow[cluster-2],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

# Clustering with respect to Population Growth(No. of Clusters = 3)

In [80]:
Gujaratpop=Gujarat_grouped3[['District','PopulationGrowth','Population2011','Density2011']]
Gujaratpop

,District,PopulationGrowth,Population2011,Density2011
0,Ahmedabad,24.1883,7045313,983
1,Amreli,8.58998,1513614,224
2,Anand,12.5794,2090276,446
3,Banaskantha,24.5002,3116045,245
4,Bharuch,13.1901,1550822,238
5,Bhavnagar,15.8693,2393272,287
6,Botad,19.8036,656005,256
7,Chhota Udaipur,17.8096,1071831,331
8,Dahod,30.035,2126558,583
9,Gandhinagar,3.95188,1387478,641


In [81]:
Gujaratpop_clustering = Gujaratpop.drop('District', 1)
Gujaratpop_clustering= preprocessing.StandardScaler().fit(Gujaratpop_clustering).transform(Gujaratpop_clustering)
Gujaratpop_clustering[0:5]

/opt/conda/envs/Python36/lib/python3.6/site-packages/sklearn/preprocessing/data.py:645: DataConversionWarning: Data with input dtype object were all converted to float64 by StandardScaler.
  return self.partial_fit(X, y)
/opt/conda/envs/Python36/lib/python3.6/site-packages/ipykernel/__main__.py:2: DataConversionWarning: Data with input dtype object were all converted to float64 by StandardScaler.
  from ipykernel import kernelapp as app


array([[ 1.42731676e+00,  3.33321103e+00,  2.02230321e+00],
       [-1.16231405e+00, -3.84919235e-01, -8.05076939e-01],
       [-4.99983344e-01,  2.68400162e-03,  2.19038146e-02],
       [ 1.47910447e+00,  6.92154410e-01, -7.26849030e-01],
       [-3.98602861e-01, -3.59909886e-01, -7.52925000e-01]])

In [82]:
kclusters=3
# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(Gujaratpop_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10]

array([1, 0, 0, 2, 0, 2, 2, 2, 2, 0], dtype=int32)

In [83]:
Gujaratpop.insert(0, 'Cluster Labels', kmeans.labels_)
Gujaratpop

,Cluster Labels,District,PopulationGrowth,Population2011,Density2011
0,1,Ahmedabad,24.1883,7045313,983
1,0,Amreli,8.58998,1513614,224
2,0,Anand,12.5794,2090276,446
3,2,Banaskantha,24.5002,3116045,245
4,0,Bharuch,13.1901,1550822,238
5,2,Bhavnagar,15.8693,2393272,287
6,2,Botad,19.8036,656005,256
7,2,Chhota Udaipur,17.8096,1071831,331
8,2,Dahod,30.035,2126558,583
9,0,Gandhinagar,3.95188,1387478,641


In [86]:
temp=Gujarat_merged4[['District','Latitude','Longitude']]
Gujaratpop = Gujaratpop.join(temp.set_index('District'), on='District')
Gujaratpop.head()

,Cluster Labels,District,PopulationGrowth,Population2011,Density2011,Latitude,Longitude
0,1,Ahmedabad,24.1883,7045313,983,23.0216,72.5797
1,0,Amreli,8.58998,1513614,224,20.8667,70.75
2,0,Anand,12.5794,2090276,446,22.5585,72.9626
3,2,Banaskantha,24.5002,3116045,245,24.1721,72.4311
4,0,Bharuch,13.1901,1550822,238,21.75,73


In [87]:
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=7)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(Gujaratpop['Latitude'], Gujaratpop['Longitude'], Gujaratpop['District'], Gujaratpop['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-3],
        fill=True,
        fill_color=rainbow[cluster-2],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

# Comparing the Results to create the Matrix

Both Population growth and Urban Development

High Cluster

In [91]:
Guj_1 = Gujarat_merged4[Gujarat_merged4['Cluster Labels'] == 0]
Guj_1

,No.,District,Population2001,Population2011,Density2011,PopulationGrowth,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,1,Ahmedabad,5673090,7045313,983,24.188282,23.0216,72.5797,0,Indian Restaurant,Café,Hotel,Fast Food Restaurant,Tea Room,Sandwich Place,Vegetarian / Vegan Restaurant,Dessert Shop,Snack Place,History Museum


Medium Cluster

In [109]:
Guj_1 = Gujarat_merged4[Gujarat_merged4['Cluster Labels'] == 2]
Guj_1

,No.,District,Population2001,Population2011,Density2011,PopulationGrowth,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
28,29,Surat,4996391,6079231,1337,21.672443,21.1865,72.8081,2,Indian Restaurant,Fast Food Restaurant,Restaurant,Pizza Place,Chinese Restaurant,Café,Ice Cream Shop,Tea Room,Sandwich Place,Juice Bar


Low Cluster

In [118]:
Guj_1 = Gujarat_merged4[Gujarat_merged4['Cluster Labels'] == 1]
Guj_1

,No.,District,Population2001,Population2011,Density2011,PopulationGrowth,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
1,2,Amreli,1393880,1513614,224,8.589979,20.8667,70.75,1,Beach,Zoo,Clothing Store,Construction & Landscaping,Cricket Ground,Dairy Store,Department Store,Dessert Shop,Diner,Electronics Store
2,3,Anand,1856712,2090276,446,12.579442,22.5585,72.9626,1,Fast Food Restaurant,Dessert Shop,Indian Restaurant,Café,Resort,Restaurant,Pizza Place,Sandwich Place,Factory,BBQ Joint
4,5,Banaskantha,2502843,3116045,245,24.500218,24.1721,72.4311,1,Indian Restaurant,Train Station,Mobile Phone Shop,Ice Cream Shop,Multiplex,Zoo,Electronics Store,Coffee Shop,Construction & Landscaping,Cricket Ground
5,6,Bharuch,1370104,1550822,238,13.190094,21.75,73,1,Hotel,Multiplex,American Restaurant,Factory,Coffee Shop,Construction & Landscaping,Cricket Ground,Dairy Store,Department Store,Dessert Shop
6,7,Bhavnagar,2065492,2393272,287,15.869343,21.7719,72.1416,1,Café,Fast Food Restaurant,Multiplex,Garden,Hotel,Pizza Place,Asian Restaurant,Shopping Mall,Diner,Coffee Shop
7,8,Botad,547567,656005,256,19.803604,22.1686,71.6685,1,Restaurant,Bakery,Chinese Restaurant,Train Station,Bus Station,Pub,Asian Restaurant,Jewelry Store,Cricket Ground,Dairy Store
8,9,Chhota Udaipur,909799,1071831,331,17.809648,22.3049,74.0126,1,ATM,Ice Cream Shop,Farmers Market,Construction & Landscaping,Cricket Ground,Dairy Store,Department Store,Dessert Shop,Diner,Electronics Store
9,10,Dahod,1635374,2126558,583,30.034964,22.8359,74.2557,1,Train Station,Motel,Indian Restaurant,Asian Restaurant,Zoo,Factory,Construction & Landscaping,Cricket Ground,Dairy Store,Department Store
12,13,Gandhinagar,1334731,1387478,641,3.951882,23.2233,72.6492,1,Indian Restaurant,Café,Pizza Place,Fast Food Restaurant,Asian Restaurant,Sandwich Place,Multiplex,Ice Cream Shop,Italian Restaurant,Indian Sweet Shop
14,15,Jamnagar,1281187,1407635,167,9.869597,22.4732,70.0552,1,Hotel,Indian Restaurant,Multiplex,Café,Smoke Shop,Fast Food Restaurant,Gas Station,Park,Pizza Place,Shopping Mall


With respect to Population alone

High Cluster(Density High, Growth Medium)

In [126]:
Guj_2 = Gujaratpop[Gujaratpop['Cluster Labels'] == 1]
Guj_2

,Cluster Labels,District,PopulationGrowth,Population2011,Density2011,Latitude,Longitude
0,1,Ahmedabad,24.1883,7045313,983,23.0216,72.5797
20,1,Surat,21.6724,6079231,1337,21.1865,72.8081


Medium Cluster(Density Low, Growth High)

In [124]:
Guj_2 = Gujaratpop[Gujaratpop['Cluster Labels'] == 2]
Guj_2

,Cluster Labels,District,PopulationGrowth,Population2011,Density2011,Latitude,Longitude
3,2,Banaskantha,24.5002,3116045,245,24.1721,72.4311
5,2,Bhavnagar,15.8693,2393272,287,21.7719,72.1416
6,2,Botad,19.8036,656005,256,22.1686,71.6685
7,2,Chhota Udaipur,17.8096,1071831,331,22.3049,74.0126
8,2,Dahod,30.035,2126558,583,22.8359,74.2557
14,2,Morbi,16.3611,960329,197,22.8177,70.8346
16,2,Panchmahal,18.9186,1642268,502,22.7731,73.6255
18,2,Rajkot,21.1478,3015229,399,22.3052,70.8028
19,2,Sabarkantha,18.3122,1388671,333,24.0295,73.046
21,2,Surendranagar,15.6418,1585268,171,22.75,71.6667


Low Cluster(Density medium, Growth Low)

In [125]:
Guj_2 = Gujaratpop[Gujaratpop['Cluster Labels'] == 0]
Guj_2

,Cluster Labels,District,PopulationGrowth,Population2011,Density2011,Latitude,Longitude
1,0,Amreli,8.58998,1513614,224,20.8667,70.75
2,0,Anand,12.5794,2090276,446,22.5585,72.9626
4,0,Bharuch,13.1901,1550822,238,21.75,73
9,0,Gandhinagar,3.95188,1387478,641,23.2233,72.6492
10,0,Jamnagar,9.8696,1407635,167,22.4732,70.0552
11,0,Junagadh,9.87448,1525605,300,21.5174,70.4643
12,0,Kheda,13.6607,2053769,560,22.75,72.8333
13,0,Mehsana,10.3407,2027727,419,23.6013,72.3742
15,0,Navsari,8.25389,1330711,602,20.9524,72.9324
17,0,Porbandar,9.16599,586062,255,21.6409,69.611


With Respect to Urban Development alone

High Cluster

In [128]:
Guj_3 = Gujarat_merged2[Gujarat_merged2['Cluster Labels'] == 0]
Guj_3

,No.,District,Population2001,Population2011,Density2011,PopulationGrowth,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,1,Ahmedabad,5673090,7045313,983,24.188282,23.0216,72.5797,0,Indian Restaurant,Café,Hotel,Fast Food Restaurant,Tea Room,Sandwich Place,Vegetarian / Vegan Restaurant,Dessert Shop,Snack Place,History Museum
2,3,Anand,1856712,2090276,446,12.579442,22.5585,72.9626,0,Fast Food Restaurant,Dessert Shop,Indian Restaurant,Café,Resort,Restaurant,Pizza Place,Sandwich Place,Factory,BBQ Joint
4,5,Banaskantha,2502843,3116045,245,24.500218,24.1721,72.4311,0,Indian Restaurant,Train Station,Mobile Phone Shop,Ice Cream Shop,Multiplex,Zoo,Electronics Store,Coffee Shop,Construction & Landscaping,Cricket Ground
5,6,Bharuch,1370104,1550822,238,13.190094,21.75,73,0,Hotel,Multiplex,American Restaurant,Factory,Coffee Shop,Construction & Landscaping,Cricket Ground,Dairy Store,Department Store,Dessert Shop
6,7,Bhavnagar,2065492,2393272,287,15.869343,21.7719,72.1416,0,Café,Fast Food Restaurant,Multiplex,Garden,Hotel,Pizza Place,Asian Restaurant,Shopping Mall,Diner,Coffee Shop
7,8,Botad,547567,656005,256,19.803604,22.1686,71.6685,0,Restaurant,Bakery,Chinese Restaurant,Train Station,Bus Station,Pub,Asian Restaurant,Jewelry Store,Cricket Ground,Dairy Store
9,10,Dahod,1635374,2126558,583,30.034964,22.8359,74.2557,0,Train Station,Motel,Indian Restaurant,Asian Restaurant,Zoo,Factory,Construction & Landscaping,Cricket Ground,Dairy Store,Department Store
12,13,Gandhinagar,1334731,1387478,641,3.951882,23.2233,72.6492,0,Indian Restaurant,Café,Pizza Place,Fast Food Restaurant,Asian Restaurant,Sandwich Place,Multiplex,Ice Cream Shop,Italian Restaurant,Indian Sweet Shop
14,15,Jamnagar,1281187,1407635,167,9.869597,22.4732,70.0552,0,Hotel,Indian Restaurant,Multiplex,Café,Smoke Shop,Fast Food Restaurant,Gas Station,Park,Pizza Place,Shopping Mall
15,16,Junagadh,1388498,1525605,300,9.874483,21.5174,70.4643,0,Zoo,Hotel,Mountain,Multiplex,City,Garden,Frozen Yogurt Shop,Coffee Shop,General Travel,Construction & Landscaping


Medium Cluster

In [132]:
Guj_3 = Gujarat_merged2[Gujarat_merged2['Cluster Labels'] == 1]
Guj_3

,No.,District,Population2001,Population2011,Density2011,PopulationGrowth,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
8,9,Chhota Udaipur,909799,1071831,331,17.809648,22.3049,74.0126,1,ATM,Ice Cream Shop,Farmers Market,Construction & Landscaping,Cricket Ground,Dairy Store,Department Store,Dessert Shop,Diner,Electronics Store
27,28,Sabarkantha,1173734,1388671,333,18.312241,24.0295,73.046,1,ATM,Historic Site,Coffee Shop,Construction & Landscaping,Cricket Ground,Dairy Store,Department Store,Dessert Shop,Diner,Electronics Store
32,33,Valsad,1410680,1703068,561,20.726742,20.4324,73.1412,1,ATM,Historic Site,Coffee Shop,Construction & Landscaping,Cricket Ground,Dairy Store,Department Store,Dessert Shop,Diner,Electronics Store


Low Cluster

In [133]:
Guj_3 = Gujarat_merged2[Gujarat_merged2['Cluster Labels'] == 2]
Guj_3

,No.,District,Population2001,Population2011,Density2011,PopulationGrowth,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
1,2,Amreli,1393880,1513614,224,8.589979,20.8667,70.75,2,Beach,Zoo,Clothing Store,Construction & Landscaping,Cricket Ground,Dairy Store,Department Store,Dessert Shop,Diner,Electronics Store


Sub-dividing the Low Cluster of Urban development and Population 

In [136]:
Guj_L = Gujarat_merged4[Gujarat_merged4['Cluster Labels'] == 1]
Guj_L= Guj_L[['District']]
Guj_L.head()

,District
1,Amreli
2,Anand
4,Banaskantha
5,Bharuch
6,Bhavnagar


In [147]:
Guj_L = Guj_L.join(Gujarat_grouped4.set_index('District'), on='District')

In [149]:
Guj_L

,District,ATM,Airport,Airport Terminal,American Restaurant,Arcade,Art Gallery,Arts & Crafts Store,Arts & Entertainment,Asian Restaurant,...,Tea Room,Theater,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Store,Zoo,Population2011,Density2011,PopulationGrowth
1,Amreli,0.0,0.000000,0.000000,0.00,0.0,0.0,0.000000,0.00,0.000000,...,0.000000,0.0,0.00,0.000000,0.000,0.0,0.0,1513614,224,8.58998
2,Anand,0.0,0.000000,0.000000,0.00,0.0,0.0,0.000000,0.00,0.050000,...,0.000000,0.0,0.00,0.000000,0.000,0.0,0.0,2090276,446,12.5794
4,Banaskantha,0.0,0.000000,0.000000,0.00,0.0,0.0,0.000000,0.00,0.000000,...,0.000000,0.0,0.00,0.166667,0.000,0.0,0.0,3116045,245,24.5002
5,Bharuch,0.0,0.000000,0.000000,0.25,0.0,0.0,0.000000,0.00,0.000000,...,0.000000,0.0,0.00,0.000000,0.000,0.0,0.0,1550822,238,13.1901
6,Bhavnagar,0.0,0.000000,0.000000,0.00,0.0,0.0,0.000000,0.00,0.100000,...,0.000000,0.0,0.00,0.000000,0.000,0.0,0.0,2393272,287,15.8693
7,Botad,0.0,0.000000,0.000000,0.00,0.0,0.0,0.000000,0.00,0.111111,...,0.000000,0.0,0.00,0.111111,0.000,0.0,0.0,656005,256,19.8036
8,Chhota Udaipur,0.5,0.000000,0.000000,0.00,0.0,0.0,0.000000,0.00,0.000000,...,0.000000,0.0,0.00,0.000000,0.000,0.0,0.0,1071831,331,17.8096
9,Dahod,0.0,0.000000,0.000000,0.00,0.0,0.0,0.000000,0.00,0.250000,...,0.000000,0.0,0.00,0.250000,0.000,0.0,0.0,2126558,583,30.035
12,Gandhinagar,0.0,0.000000,0.000000,0.00,0.0,0.0,0.000000,0.04,0.080000,...,0.000000,0.0,0.04,0.000000,0.000,0.0,0.0,1387478,641,3.95188
14,Jamnagar,0.0,0.000000,0.052632,0.00,0.0,0.0,0.000000,0.00,0.000000,...,0.000000,0.0,0.00,0.052632,0.000,0.0,0.0,1407635,167,9.8696


In [167]:
Guj_L1=Guj_L.copy()
Guj_L.drop([1,8,27,32], inplace = True )# Removing LLL and MML clusters
GujaratL_clustering = Guj_L.drop('District', 1)
GujaratL_clustering
GujaratL_clustering= preprocessing.StandardScaler().fit(GujaratL_clustering).transform(GujaratL_clustering)
GujaratL_clustering[0:5]

/opt/conda/envs/Python36/lib/python3.6/site-packages/sklearn/preprocessing/data.py:645: DataConversionWarning: Data with input dtype float64, object were all converted to float64 by StandardScaler.
  return self.partial_fit(X, y)
/opt/conda/envs/Python36/lib/python3.6/site-packages/ipykernel/__main__.py:5: DataConversionWarning: Data with input dtype float64, object were all converted to float64 by StandardScaler.


array([[ 0.        , -0.30627005, -0.23570226, -0.23570226,  0.        ,
         0.        , -0.23570226, -0.23570226, -0.16246735,  0.        ,
         3.56293151, -0.36195466, -0.23570226, -0.23570226,  0.        ,
         0.        , -0.34233585, -0.23570226, -0.23570226,  1.16073079,
        -0.27791325, -0.23570226, -0.23570226,  2.81216052, -0.23570226,
         0.        ,  0.        , -0.23570226,  4.04936177,  0.        ,
        -0.23570226,  4.24264069,  0.        ,  1.3302075 , -0.23570226,
        -0.23570226, -0.23570226,  0.        , -0.23570226, -0.23570226,
        -0.23570226, -0.23570226, -0.23570226, -0.23570226, -0.23570226,
        -0.23570226, -0.54125883, -0.23570226,  0.40647682, -0.20673506,
        -0.23570226, -0.34291737, -0.23570226, -0.23570226, -0.23570226,
         0.        , -0.23570226, -0.23570226, -0.23570226, -0.23570226,
         0.        , -0.23570226, -0.23570226,  3.57387247, -0.08667642,
        -0.29194574,  0.        ,  0.12148734, -0.2

In [168]:
kclusters=2
# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(GujaratL_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_

array([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0],
      dtype=int32)

In [169]:
Guj_L.insert(0, 'Cluster Labels', kmeans.labels_)

Guj_L

,Cluster Labels,District,ATM,Airport,Airport Terminal,American Restaurant,Arcade,Art Gallery,Arts & Crafts Store,Arts & Entertainment,...,Tea Room,Theater,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Store,Zoo,Population2011,Density2011,PopulationGrowth
2,1,Anand,0.0,0.000000,0.000000,0.00,0.0,0.0,0.000000,0.00,...,0.000000,0.0,0.00,0.000000,0.000,0.0,0.0,2090276,446,12.5794
4,1,Banaskantha,0.0,0.000000,0.000000,0.00,0.0,0.0,0.000000,0.00,...,0.000000,0.0,0.00,0.166667,0.000,0.0,0.0,3116045,245,24.5002
5,1,Bharuch,0.0,0.000000,0.000000,0.25,0.0,0.0,0.000000,0.00,...,0.000000,0.0,0.00,0.000000,0.000,0.0,0.0,1550822,238,13.1901
6,1,Bhavnagar,0.0,0.000000,0.000000,0.00,0.0,0.0,0.000000,0.00,...,0.000000,0.0,0.00,0.000000,0.000,0.0,0.0,2393272,287,15.8693
7,1,Botad,0.0,0.000000,0.000000,0.00,0.0,0.0,0.000000,0.00,...,0.000000,0.0,0.00,0.111111,0.000,0.0,0.0,656005,256,19.8036
9,1,Dahod,0.0,0.000000,0.000000,0.00,0.0,0.0,0.000000,0.00,...,0.000000,0.0,0.00,0.250000,0.000,0.0,0.0,2126558,583,30.035
12,1,Gandhinagar,0.0,0.000000,0.000000,0.00,0.0,0.0,0.000000,0.04,...,0.000000,0.0,0.04,0.000000,0.000,0.0,0.0,1387478,641,3.95188
14,1,Jamnagar,0.0,0.000000,0.052632,0.00,0.0,0.0,0.000000,0.00,...,0.000000,0.0,0.00,0.052632,0.000,0.0,0.0,1407635,167,9.8696
15,1,Junagadh,0.0,0.000000,0.000000,0.00,0.0,0.0,0.000000,0.00,...,0.000000,0.0,0.00,0.000000,0.000,0.0,0.2,1525605,300,9.87448
17,1,Kheda,0.0,0.000000,0.000000,0.00,0.0,0.0,0.000000,0.00,...,0.000000,0.0,0.00,0.000000,0.000,0.0,0.0,2053769,560,13.6607


Hence, Vadodara moves to a Higher cluster in the Low cluster of both Urban and Population Development 